## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula
from datetime import datetime, timedelta

## Read-in

In [2]:
df = pd.read_csv('Houston 9% LIHTC proposed developments - Sheet1.csv')

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [3]:
df.columns

Index(['Developer', 'Project Name', 'Address/Location', 'Total Units',
       'Target Population', 'Council District'],
      dtype='object')

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address/Location'].apply(geocode)

In [6]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [7]:
# df.at[index#,'col_name']

## Format Data

In [7]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

## HTML Popup Formatter

In [8]:
df.columns

Index(['Developer', 'Project Name', 'Address/Location', 'Total Units',
       'Target Population', 'Council District', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [10]:
def popup_html(row):
    developer = row['Developer']
    name = row['Project Name']
    address = row['Address/Location']
    num_units = row['Total Units']
    target_pop = row['Target Population']
    council_dist = row['Council District']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Developer: </strong>{}'''.format(developer) + '''<br>
    <strong>Project Name: </strong>{}'''.format(name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Number of Units: </strong>{}'''.format(num_units) + '''<br>
    <strong>Target Population: </strong>{}'''.format(target_pop) + '''<br>
    <strong>Council District: </strong>{}'''.format(council_dist) + '''<br>
    </html>
    '''
    return html

In [11]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9,tiles=None)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Houston 9% LIHTC proposed developments')


for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)


folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [12]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [13]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Houston_9_percent_LIHTC
